# Libs

In [1]:
from itertools import combinations
import random
from abc import ABC, abstractmethod 
import copy
import numpy as np
import matplotlib.pyplot as plt
import time 

# Constants

In [2]:
NUMBER_OF_ITERATIONS = 100
NUMBER_OF_PARTICLES = 60
NUMBER_OF_CITIES = 12 
SAMPLE_RATE = 2

# Generate cities

In [3]:
cities = []
x = 0
y = 0
for i in range(NUMBER_OF_CITIES):
  if i % 2 == 0:
    cities.append(("", (x,y)))
  else:
    cities.append(("", (x, y+1)))
    x += 1

# Utils

In [4]:

x = []
y = []

def fitness(state):
    total = 0
    for i in range(len(state)-1):
        a = state[i+1] - 1
        b = state[i] - 1
        cityPosition1 = cities[a][1]
        cityPosition2 = cities[b][1]
        dist = (cityPosition1[0] - cityPosition2[0])**2 + (cityPosition1[1] - cityPosition2[1])**2
        total += dist 
    a = state[0] - 1
    b = state[-1] - 1
    cityPosition1 = cities[a][1]
    cityPosition2 = cities[b][1]
    dist = (cityPosition1[0] - cityPosition2[0])**2 + (cityPosition1[1] - cityPosition2[1])**2
    total += dist
    return total

def generateRandomPosition(N):
    l = [c+1 for c in range(N)]
    position = []
    for j in range(N):
        indexChosen = random.randint(0,len(l)-1)
        position.append(l[indexChosen])
        del l[indexChosen]
    return position

def generateRandomSwap(N):
    numOfSwaps = random.randint(1,N//2)
    swaps = []
    for _ in range(numOfSwaps):
        i = random.randint(0,N-1)
        j = random.randint(0,N-1)
        swaps.append((i,j))
    return swaps



# Position and velocity

In [5]:
class DiscreteDimension(ABC):
    def __init__(self):
        pass
        type = None
        values = None
    
    def __add__(self, other):
        if self.type == 'position' and other.type == 'velocity':
            returnObj = copy.deepcopy(self)
            for swap in other.values:
                i,j = swap
                returnObj.values[i], returnObj.values[j] = returnObj.values[j], returnObj.values[i]
            return returnObj
        
        if self.type == 'velocity' and other.type == 'position':
            returnObj = copy.deepcopy(other)
            for swap in self.values:
                i,j = swap
                returnObj.values[i], returnObj.values[j] = returnObj.values[j], returnObj.values[i]
            return returnObj
        
        if self.type == 'velocity' and other.type == 'velocity':
            returnObj = copy.deepcopy(self)
            returnObj.values.extend(other.values)
            return returnObj
        
        print("DiscreteDimension(Class): operation not specified")
        return 
    
    def __sub__(self, other):
        if self.type == 'position' and self.type == 'position':
            returnObj = copy.deepcopy(self)
            returnObj.values = []
            returnObj.type = 'velocity'
            auxObj = copy.deepcopy(other)
            for i in range(len(self.values)):
                if auxObj.values[i] != self.values[i]:
                    j = auxObj.values.index(self.values[i])
                    swap = (i,j)
                    returnObj.values.append(swap)
                    auxObj.values[i], auxObj.values[j] = auxObj.values[j], auxObj.values[i]
            return returnObj

    def __rmul__(self, other):
        if self.type == 'velocity':
            if other < 0:
                print("DiscreteDimension(Class): operation not specified: multiplication by negative number")
                return 
            if other > 1:
                returnObj = copy.deepcopy(self)
                integerPart = int(other)
                fracPart = int(other*10)%10
                returnObj.values = returnObj.values * integerPart
                returnObj.values.extend(self.values[:fracPart])
                return returnObj
            if other == 0:
                returnObj = copy.deepcopy(self)
                returnObj.values = []
                return returnObj
            
            fracPart = int(other*10)%10
            returnObj = copy.deepcopy(self)
            returnObj.values = returnObj.values[:fracPart]
            return returnObj
    
    def __str__(self):
        return f"{self.values}, type={self.type}"


class Position(DiscreteDimension):
    def __init__(self):
        DiscreteDimension.__init__(self)
        self.type = 'position'
        self.values = []

class Velocity(DiscreteDimension):
    def __init__(self):
        DiscreteDimension.__init__(self)
        self.type = 'velocity'
        self.values = []



# Particle

In [6]:
class Particle:
    def __init__(self, N):
        self.N = N
        self.position = Position()
        self.position.values = generateRandomPosition(N)
        self.velocity = Velocity()
        self.velocity.values = generateRandomSwap(N)
        self.pbest = (self.position, fitness(self.position.values))
        self.omega = 0.3
        self.alfa = 1.1
        self.beta = 2.5
    
    def update(self, gbest):
        self.r1 = random.uniform(0,1)
        self.r2 = random.uniform(0,1)
        
        inertiaFactor = self.omega*self.velocity
        cognitiveFactor = self.alfa*self.r1*(self.pbest[0] - self.position)
        socialFactor = self.beta*self.r2*(gbest - self.position)
        
        self.velocity = inertiaFactor + cognitiveFactor + socialFactor
        self.position = self.position + self.velocity

        if fitness(self.position.values) < self.pbest[1]:
            self.pbest = (self.position, fitness(self.position.values))

# TSPSolver

In [7]:

class TSPSolver:
    def __init__(self, numberOfCities):
        self.solution = None
        self.numberOfParticles = NUMBER_OF_PARTICLES
        self.swarm = [Particle(numberOfCities) for _ in range(self.numberOfParticles)]
        self.iteration = 0

    def solve(self):
        gBestPosition = Position()
        gBestPosition.values = self.swarm[0].position.values
        gbest = (gBestPosition, self.swarm[0].pbest[1])
        gbest = self.updateGBest(gbest)

        i = 0
        while i < NUMBER_OF_ITERATIONS:
            self.iteration += 1
            i += 1
            for p in self.swarm:
                p.update(gbest[0])
            gbest = self.updateGBest(gbest)
            if self.iteration % SAMPLE_RATE == 0:
              x.append(self.iteration)
              y.append(gbest[1])
        
        self.solution = gbest[0].values
        #print(gbest[1], self.iteration, self.solution)
        #plt.plot(x,y)
    
    def converged(self, gbest):
      majority = len(self.swarm) // 2 + 1
      gbest_v = gbest[1]
      cont = 0
      for i in self.swarm:
        if i.pbest[1] == gbest_v:
          cont += 1
      return cont >= majority and gbest_v == NUMBER_OF_CITIES
    
    def updateGBest(self, gbestValue):
        gbest = gbestValue
        for p in self.swarm:
            if p.pbest[1] < gbest[1]:
                gbest = (p.pbest[0],p.pbest[1])
        
        return gbest
    
    def getSolution(self):
        return self.solution



# Main

In [8]:
# Generate TSPSolver object
tsp = TSPSolver(NUMBER_OF_CITIES)

# Start timer
start = time.time()

# Run 100 iterations
tsp.solve()

# Finish timer
end = time.time()

# Show results 
print("Time taked to run 100 iterations:", end - start)

Time taked to run 100 iterations: 1.7058019638061523
